In [ ]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import time
import pymongo

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [ ]:
url = 'https://ark.intel.com/content/www/us/en/ark.html#@Processors'
baseurl = 'https://ark.intel.com/'

browser.visit(url)

rootHTML = bs(browser.html, 'html.parser')

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.IntelScrape
collection = db.items

In [ ]:
for link in bs(browser.html, 'html.parser').\
    find_all('div', {'data-parent-panel-key':'PanelLabel122139'})[0].\
    find_all('a'):
    browser.visit(baseurl+link['href'])
    time.sleep(1)
    for td in bs(browser.html, 'html.parser').find_all('td', class_='ark-product-name'):
        browser.visit(baseurl+td.a['href'])
        time.sleep(1)
        data = {}
        for c in bs(browser.html, 'html.parser').\
            find_all('div', class_='arkProductSpecifications')[0].\
            find_all('span', class_=['label','value']):
            if c['class'][0] == 'label':
                temp = c.text.strip().replace('.','_')
            else:
                data[temp] = c.text.strip()
        collection.insert_one(data)

# alternate 
span = bs(browser.html, 'html.parser')
data = dict(zip(
[label.text.strip() for label in span.find_all('div', class_='arkProductSpecifications')[0].find_all('span', class_='label')],
[value.text.strip() for value in span.find_all('div', class_='arkProductSpecifications')[0].find_all('span', class_='value')]
))

In [ ]:
browser.quit()

In [ ]:
import csv
with open('IntelArk.csv','w',newline='') as csvfile:
    cols = []
    for i in db.items.find():
        for k in i.keys():
            if k in cols:
                pass
            else:
                cols.append(k)
    writer = csv.DictWriter(csvfile, cols)
    writer.writeheader()
    for i in db.items.find():
        writer.writerow(i)